On enterprise, testing different short read simulators.

Good article about whole genome sequencing process from lab to assembly.
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4231593/

Good pub listing a bunch of different short read simulators:
http://www.nature.com.ezproxy.gc.cuny.edu/nrg/journal/v17/n8/full/nrg.2016.57.html#sequencing-depth

Timing for 1M PE reads

# Estimate reference size from illumina reads

http://josephryan.github.io/estimate_genome_size.pl/

Thoughts on how to deal w/ low coverage:
http://seqanswers.com/forums/showthread.php?t=41874

`jellyfish count -m 31 -o fastq.counts -C -s 1000000000 -U 500 -t 30 ~/able/frog_locov/Sample_16_116_read1_trimmomatic_1PE.fastq ~/able/frog_locov/Sample_16_116_read2_trimmomatic_2PE.fastq`

`jellyfish histo fastq.counts_0 > fastq.counts_0.histo`

`./find_valleys.pl fastq.counts.histo`
`./jellyplot.pl fastq.counts.histo`

# Quantify quality of genome assembly

http://busco.ezlab.org/ - Searches assembly for universal single copy orthologs to assess assembly quality. Looks like a bitch to set up and takes a long time to run (~1 Wk for human genome).


# Generate simulated whole genome sequence

## ARGON
https://github.com/pierpal/ARGON
Fast, whole-genome simulation of the discrete time Wright-Fisher process


# Simulate illumina reads from a reference

## pIRS
https://github.com/galaxy001/pirs

## wgsim
https://github.com/lh3/wgsim
hung after writing only a fraction of the reads.


## simNGS
https://www.biostars.org/p/8752/

`simLibrary -n [number_of_fragments] reference.fas | simNGS -o fastq -p paired [runfile] > simulated_reads.fq`

Requires a run file that I don't have.

## simhtsd
perl script. Requires bioperl install.

Runs fast, but doesn't introduce variation, except for an optional error rate.
`./simhtsd.pl -o simht_out -n 1000000 -p 2 -l 400 -s 10 -r 100 ~/able/xenopus/Xla.v91.repeatMasked.fa`

## ART
`art_illumina -ss HS25 -sam -i ~/able/xenopus/Xla.v91.repeatMasked.fa -c 10000 -p -l 150 -m 200 -s 10 -o paired_dat`


# Genome assembly

An example of assembly and mapping: https://academic-oup-com.ezproxy.gc.cuny.edu/gbe/article/9/4/1084/3610021/Genomic-Differentiation-and-Demographic-Histories

## SOAP
http://soap.genomics.org.cn/soapdenovo.html

`SOAPdenovo-63mer all -s conf.txt -K 33  -R -o prefix 1> out.log 2> err.log`

SOAP requires a conf file which looks like this:
```
#maximal read length
max_rd_len=150
[LIB]
#average insert size
avg_ins=300
#if sequence needs to be reversed
reverse_seq=0
#in which part(s) the reads are used
asm_flags=3
#use only first 100 bps of each read
rd_len_cutoff=150
#in which order the reads are used while scaffolding
rank=1
# cutoff of pair number for a reliable connection (at least 3 for short insert size)
pair_num_cutoff=3
#minimum aligned length to contigs for a reliable read location (at least 32 for short insert size)
map_len=32
#a pair of fastq file, read 1 file should always be followed by read 2 file
q1=/home/iovercast/able/frog_locov/Sample_16_116_read1_trimmomatic_1PE.fastq
q2=/home/iovercast/able/frog_locov/Sample_16_116_read2_trimmomatic_2PE.fastq
```

## Abyss

http://computing.bio.cam.ac.uk/local/doc/abyss.html#compilingabyssfromsource

Output file formats: https://github.com/bcgsc/abyss/wiki/ABySS-File-Formats

`abyss-pe name=abyss k=31 in="../frog_locov/Sample_16_116_read1_trimmomatic_1PE.fastq ../frog_locov/Sample_16_116_read2_trimmomatic_2PE.fastq"`

Example - abyss-pe k=64 name=ecoli lib='lib1 lib2' \
            lib1='lib1_1.fa lib1_2.fa' lib2='lib2_1.fa lib2_2.fa' \
            se='se1.fa se2.fa'

abyss-pe name=abyss k=31 lib="pe1" pe1="/Volumes/Glyph2TB/raws/High_Coverage_Genomes/Ptychadena_15-104/Sample_15-104_read1_trimmomatic_1PE /Volumes/Glyph2TB/raws/High_Coverage_Genomes/Ptychadena_15-104/Sample_15-104_read2_trimmomatic_2PE" se="/Volumes/Glyph2TB/raws/High_Coverage_Genomes/Ptychadena_15-104/Sample_15-104_read1_trimmomatic_1SE /Volumes/Glyph2TB/raws/High_Coverage_Genomes/Ptychadena_15-104/Sample_15-104_read2_trimmomatic_2SE" 

## SGA

https://github.com/jts/sga/wiki
http://genome.cshlp.org.ezproxy.gc.cuny.edu/content/22/3/549.full

Supposed to be fast.

-----------------

## minia
More of a toy. Runs fast. Doesn't make scaffold, just contigs.

## AllPaths-LG
Requires very high coverage (100x) and at least 2 libraries (1 w/ short and 1 w/ long insert sizes). This _may_ not be true, but it's still true that it's a bitch to configure and run.
ftp://ftp.broadinstitute.org/pub/crd/ALLPATHS/Release-LG/AllPaths-LG_Manual.pdf

# Mapping

`bwa mem prefix.scafSeq ../frog_locov/Sample_16_116_read1_trimmomatic_1PE.fastq ../frog_locov/Sample_16_116_read2_trimmomatic_2PE.fastq > tmp.sam`

# Estimating Average coverage
Useful: https://www.biostars.org/p/104063/ and more of the same: https://www.biostars.org/p/5165/

http://bedtools.readthedocs.io/en/latest/content/tools/genomecov.html

`bedtools genomecov -ibam tmp.bam -g prefix.scafSeq.fai -d > genomecov.out`

# Genotype likelihoods
Proto-angsd paper: http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0037558
ANGSD paper: https://bmcbioinformatics-biomedcentral-com.ezproxy.gc.cuny.edu/articles/10.1186/s12859-014-0356-4

ANGSD demo tutorial: https://github.com/mfumagalli/Copenhagen2016Tutorial/blob/master/DemoTutorial.md
NGSTools tutorial: https://github.com/mfumagalli/ngsTools/blob/master/TUTORIAL.md
ANGSD-Wrapper: https://github.com/mojaveazure/angsd-wrapper

In [1]:
import msprime